In [1]:
bin_dir = './'

In [2]:
import sys
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout
import pandas as pd
import networkx as nx
import seaborn as sns
sys.path.append('..')
import conet 
import conet.src.data_converter.data_converter as dc
import conet.src.conet as c
import conet.src.conet_parameters as cp
import conet.src.inference_result as ir

In [3]:
data_converter = dc.DataConverter("data/SA501X3F_filtered_corrected_counts.csv", ',', 150000, 3095677412)

In [4]:
breakpoint_candidates_indices = pd.read_csv('data/indices.csv', header=None, sep = ' ')[1].tolist()
breakpoint_candidates_indices = list(map(lambda x : x - 1, breakpoint_candidates_indices))

In [17]:
data_converter.create_CoNET_input_files(breakpoint_candidates_indices, bin_dir, chromosomes=[17,18, 20, 23], add_chr_ends_to_indices=True)

In [20]:
conet = c.CoNET(bin_dir + "CoNET")
params = cp.CoNETParameters(data_size_prior_c = 0.3, data_dir = bin_dir, counts_penalty_c=200000, param_inf_iters=50000)
conet.infer_tree(params)

./CoNET ./ 50000 100000 200000 1.0 0.3 1 12312 4 5 4 10 10 10000 1
b'Input files have been loaded succesfully\n'
b'Starting parameter estimation\n'
b'Finished parameter estimation\n'
b'Estimated breakpoint distribution:\n'
b'(weight: 1 mean: 0.101568sd: 0.412598)\n'
b'\n'
b'Estimated no-breakpoint distribution:\n'
b'mean: 0 sd: 0.340384\n'
b'\n'
b'State after 0 iterations:\n'
b'Tree size: 5\n'
b'Log-likelihood: 8877.59\n'
b'State after 10000 iterations:\n'
b'Tree size: 25\n'
b'Log-likelihood: 10239\n'
b'State after 20000 iterations:\n'
b'Tree size: 26\n'
b'Log-likelihood: 10295\n'
b'State after 30000 iterations:\n'
b'Tree size: 28\n'
b'Log-likelihood: 10317.7\n'
b'State after 40000 iterations:\n'
b'Tree size: 27\n'
b'Log-likelihood: 10307\n'
b'State after 50000 iterations:\n'
b'Tree size: 27\n'
b'Log-likelihood: 10302.7\n'
b'State after 60000 iterations:\n'
b'Tree size: 30\n'
b'Log-likelihood: 10737.6\n'
b'State after 70000 iterations:\n'
b'Tree size: 30\n'
b'Log-likelihood: 10735.4\n'